In [8]:
!nvcc --version
%env OMP_NUM_THREADS=3

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
env: OMP_NUM_THREADS=3


In [21]:
%%writefile bubble.cpp

#include <iostream>
#include <omp.h>

using namespace std;

void sequentialBubbleSort(int *, int);
void parallelBubbleSort(int *, int);
void swap(int &, int &);

void sequentialBubbleSort(int *a, int n)
{
    int swapped;
    for (int i = 0; i < n; i++)
    {
        swapped = 0;
        for (int j = 0; j < n - 1; j++)
        {
            if (a[j] > a[j + 1])
            {
                swap(a[j], a[j + 1]);
                swapped = 1;
            }
        }

        if (!swapped)
            break;
    }
}

void parallelBubbleSort(int *a, int n)
{
    int swapped;
    for (int i = 0; i < n; i++)
    {
        swapped = 0;
        int first = i % 2;
        #pragma omp parallel for shared(a, first)
        for (int j = first; j < n - 1; j++)
        {
            if (a[j] > a[j + 1])
            {
                swap(a[j], a[j + 1]);
                swapped = 1;
            }
        }

        if (!swapped)
            break;
    }
}

void swap(int &a, int &b)
{
    int test;
    test = a;
    a = b;
    b = test;
}

int main()
{
    int *a, n;
    cout << "\nEnter total number of elements for bubble sort: ";
    cin >> n;
    a = new int[n];

    for (int i = 0; i < n; i++)
    {
        a[i] = rand() % 100; // Generate random numbers between 0 and 99
    }
    for (int i = 0; i < n; ++i){
      cout<<a[i]<<" ";
    }
    cout<<endl;


    // Perform sequential bubble sort
    double start_time = omp_get_wtime();
    sequentialBubbleSort(a, n);
    double end_time = omp_get_wtime();
    cout << "\nSorted array after sequential bubble sort:\n";
    for (int i = 0; i < n; i++)
    {
        cout << a[i] << " ";
    }
    cout << "\nTime taken by sequential bubble sort: " << end_time - start_time << " seconds" << endl;

    // Perform parallel bubble sort
    start_time = omp_get_wtime();
    parallelBubbleSort(a, n);
    end_time = omp_get_wtime();
    cout << "\nSorted array after parallel bubble sort:\n";
    for (int i = 0; i < n; i++)
    {
        cout << a[i] << " ";
    }
    cout << "\nTime taken by parallel bubble sort: " << end_time - start_time << " seconds" << endl;

    delete[] a; // Don't forget to free the allocated memory

    return 0;
}

Writing bubble.cpp


In [22]:
!g++ -fopenmp bubble.cpp -o bubble

In [23]:
!./bubble


Enter total number of elements for bubble sort: 10
83 86 77 15 93 35 86 92 49 21 

Sorted array after sequential bubble sort:
15 21 35 49 77 83 86 86 92 93 
Time taken by sequential bubble sort: 1.935e-06 seconds

Sorted array after parallel bubble sort:
15 21 35 49 77 83 86 86 92 93 
Time taken by parallel bubble sort: 0.000197334 seconds
